# Load dataset

In [19]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import os.path
from os import path

In [20]:
data = pd.read_csv('orders.csv', sep='|', low_memory=False, encoding = 'ISO-8859-1', header=None)

if path.exists('orders_new.csv'):
    data_new = pd.read_csv('orders_new.csv', sep='|', low_memory=False, encoding = 'ISO-8859-1', header=None)
    data = pd.concat([data,data_new])

## Set headers

In [21]:
del data[15]
data.columns = ['Customer_Number', 'Date_of_Birth', 'Gender', 'Place_of_Residence', 'Order_Number', 'Order_Date', 'Product_Number', 'Sub_Category', 'Category', 'Quantity', 'Price', 'Expected_Delivery_Time', 'Actual_Delivery_Time', 'Reason_of_Return', 'Rating']


In [22]:
data.tail(10)

,Customer_Number,Date_of_Birth,Gender,Place_of_Residence,Order_Number,Order_Date,Product_Number,Sub_Category,Category,Quantity,Price,Expected_Delivery_Time,Actual_Delivery_Time,Reason_of_Return,Rating
411192,1038296,1973-11-30,Man,Leek,1971817.0,24-06-2018,2182410283728,Insects and vermin,Control,1.0,247.42,3.0,8.0,NaN,2.0
411193,1070997,1963-11-09,Man,Apeldoorn,1844636.0,4-02-2018,7162326162039,Lounge sofas,Lounge furniture,1.0,687.93,4.0,7.0,NaN,-1.0
411194,1007275,1976-09-15,Woman,Dronten,2040296.0,27-09-2018,3112530283253,Insects and vermin,Control,1.0,137.39,6.0,11.0,NaN,-1.0
411195,1059348,1981-12-23,Man,Ede,1856594.0,16-02-2018,8195883096096,Garden chairs,Garden furniture,1.0,253.09,5.0,9.0,NaN,-1.0
411196,1026487,1966-03-25,Man,Rheden,1827245.0,20-01-2018,4197767134393,Garden statues,Garden decoration,1.0,332.87,1.0,5.0,NaN,3.0
411197,1038296,1973-11-30,Man,Leek,1971817.0,24-06-2018,5196143117899,Garden gnomes,Garden decoration,3.0,49.68,5.0,5.0,NaN,2.0
411198,1005692,1972-03-29,Man,Ede,1837029.0,30-01-2018,3131909207607,Insects and vermin,Control,1.0,152.46,12.0,12.0,NaN,-1.0
411199,1063650,1976-10-08,Man,Vlagtwedde,1834853.0,28-01-2018,1151665250459,Pumps,Watering,1.0,790.48,2.0,2.0,NaN,-1.0
411200,1059348,1981-12-23,Man,Ede,1856594.0,16-02-2018,5136408180381,Sunshades,Sun protection,1.0,408.77,3.0,9.0,NaN,-1.0
411201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Size

In [23]:
len(data)

3016184

## Types

In [24]:
data.dtypes

Customer_Number            object
Date_of_Birth              object
Gender                     object
Place_of_Residence         object
Order_Number              float64
Order_Date                 object
Product_Number             object
Sub_Category               object
Category                   object
Quantity                  float64
Price                     float64
Expected_Delivery_Time    float64
Actual_Delivery_Time      float64
Reason_of_Return           object
Rating                    float64
dtype: object

# Cleaning

In [25]:
data = data.drop(data[(data['Customer_Number'].isnull()) & (data['Date_of_Birth'].isnull()) & (data['Gender'].isnull())].index)

In [26]:
data['Customer_Number'] = data[['Customer_Number']].apply(pd.to_numeric, errors='coerce')

data['Order_Number'] = data['Order_Number'].astype(int)
data['Quantity'] = data['Quantity'].astype(int)
data['Expected_Delivery_Time'] = data['Expected_Delivery_Time'].astype(int)
data['Actual_Delivery_Time'] = data['Actual_Delivery_Time'].astype(int)
data['Rating'] = data['Rating'].astype(int)

data['Date_of_Birth'] = pd.to_datetime(data['Date_of_Birth'])
data['Order_Date'] = pd.to_datetime(data['Order_Date'])

data['Is_Returned'] = np.where(data['Actual_Delivery_Time'] < 0, True, False)
data['Has_Return_Reason'] = np.where((data['Reason_of_Return'].isnull()) | (data['Reason_of_Return'] == 'No reason given'), False, True)
data['Has_Rated'] = np.where(data['Rating'] >= 0, True, False)

data.head()

,Customer_Number,Date_of_Birth,Gender,Place_of_Residence,Order_Number,Order_Date,Product_Number,Sub_Category,Category,Quantity,Price,Expected_Delivery_Time,Actual_Delivery_Time,Reason_of_Return,Rating,Is_Returned,Has_Return_Reason,Has_Rated
0,NaN,1984-09-15,Man,Loppersum,1606586,2016-07-12,5183337092063,Garden sets,Garden furniture,1,1049.97,2,2,NaN,2,False,False,True
1,1049989.0,1969-06-01,Man,Goeree-Overflakkee,554370,2016-01-15,4174149044532,Garden tables,Garden furniture,1,366.90,8,8,NaN,-1,False,False,False
2,1068587.0,1970-02-19,Man,Terneuzen,1146785,2014-05-16,7167356147773,Garden statues,Garden decoration,1,493.95,2,2,NaN,-1,False,False,False
3,1026863.0,1972-11-27,Man,Sittard-Geleen,1146785,2014-05-14,4112354045616,Garden sets,Garden furniture,1,234.74,2,5,NaN,-1,False,False,False
4,1049989.0,1969-06-01,Man,Goeree-Overflakkee,554370,2016-01-15,3163858019499,Garden sets,Garden furniture,1,1436.62,3,8,NaN,-1,False,False,False


In [27]:
data['Gender'] = data.apply(
    lambda row:
        row['Gender']
        if (row['Gender'] == 'Man' or row['Gender'] == 'Woman') else 'Other',
    axis = 1)

In [28]:
data['Reason_of_Return'].unique()

array([nan, "Don't like the article", 'Wrong or missing article',
       'Disapointing quality', 'Article is defect',
       "Doesn't meet expectations", 'No reason given',
       'Article seems different than online', 'Delivery took to long',
       'Damaged package', 'Manufacturing error',
       'Ordered the wrong article by accident', 'Article is defetc',
       'Dont like the article', "Deosn't meet expectations",
       'Doesnt meet expectations', 'Artilce seems different than online',
       "Doesn't meet expetcations", 'Artilce is defect',
       'Manufatcuring error'], dtype=object)

In [29]:
def fillDerp(derp):
    if derp == 'Article is defetc':
        return 'Article is defect'
    elif derp == "Dont like the article":
        return "Don't like the article"
    elif derp == "Deosn't meet expectations":
        return "Doesn't meet expectations"
    elif derp == "Doesnt meet expectations":
        return "Doesn't meet expectations"
    elif derp == "Artilce seems different than online":
        return 'Article seems different than online'
    elif derp == "Doesn't meet expetcations":
        return "Doesn't meet expectations"
    elif derp == "Artilce is defect":
        return "Article is defect"
    elif derp == "Manufatcuring error":
        return 'Manufacturing error'
    else:
        return derp

data['Reason_of_Return'] = data['Reason_of_Return'].apply(
    lambda row:
        fillDerp(row)
        if pd.notna(row) else np.nan
)

In [30]:
print('Returned: %2.2f' % (len(data[data['Is_Returned'] == True]) / len(data) * 100) + '%')
print('Returned with a return rating: %2.2f' % (len(data[data['Has_Return_Reason'] == True]) / len(data[data['Is_Returned'] == True]) * 100) + '%')
print('With a rating: %2.2f' % (len(data[data['Has_Rated'] == True]) / len(data) * 100) + '%')
print('With a price of 0 or lower: %1d' % (len(data[data['Price'] <= 0])))
print('Other gender: %1d' % (len(data[data['Gender'] == 'Other'])))

Returned: 5.56%
Returned with a return rating: 55.05%
With a rating: 28.21%
With a price of 0 or lower: 0
Other gender: 573


In [31]:
count_nan = len(data) - data.count()
print(count_nan)

Customer_Number                 2
Date_of_Birth                   0
Gender                          0
Place_of_Residence              0
Order_Number                    0
Order_Date                      0
Product_Number                  0
Sub_Category                    0
Category                        0
Quantity                        0
Price                           0
Expected_Delivery_Time          0
Actual_Delivery_Time            0
Reason_of_Return          2880708
Rating                          0
Is_Returned                     0
Has_Return_Reason               0
Has_Rated                       0
dtype: int64


# Saving the cleaned dataset

In [32]:
data.to_csv('orders_cleaned.csv')